# Gene target analysis

Cell factory engineering strategies can be used to improve the production of progesterone in our modelled S. cerevisiae strain (iMM904_progesterone.xml).

We investigate if gene knockouts suggested in literature actually have an improving effect on the production of progesterone, and if it would be appropiate to incorporate these gene knockouts in our model.

### Knockout analysis

It has been suggested to knockout ERG5 and ERG6 in order to drive the flux towards the production of cholesterol instead of the natural produced steriod ergosterol (Figure 1) (Jiang, Yi-qi, and Jian-ping Lin 2022). It is from zymosterol, the knockouts should improve the flux towards cholesterol. It might improve the production of progesterone because cholesterol is an intermediate in the implemented heterologous pathway. However, it can be problematic to knockout ERG5 and ERG6 because ergosterol is an essential compound in the cellular membrane (Jordá, Tania, and Sergi Puig 2020).

![figures/3_Fig1_Steriods_ERG5_ERG6.png](figures/3_Fig1_Steriods_ERG5_ERG6.png)

Figure 1 (Jiang, Yi-qi, and Jian-ping Lin 2022)

A Synthesis of steroids. Solid arrows represent reactions validated in yeast, and dashed arrows implicate conversions catalyzed by 
unknown enzyme(s). The endogenous ergosterol pathway in yeast is highlighted in blue. The three different sheds of blue represented three modules: mevalonate (MVA) biosynthesis, farnesyl pyrophosphate (FPP) biosynthesis and ergosterol biosynthesis. Heterologous synthesis of animal-derived steroids in yeast already or potentially is highlighted in orange. Heterologous synthesis of phytosterols already or potentially in yeast is highlighted in green. The reaction catalyzed by UV-light is highlighted in yellow and common steroids involved in biotransformation is highlighted in purple. The enzymes marked in red represent the ones that require NADPH/NADP+, and the ones that require ATP are marked in purple. 

B The numbering of steroid’s carbon skeleton

We investigate the influence it has on the cell to knockout ERG5 and ERG6.

In [120]:
# Load libraries
from cobra.io import read_sbml_model

In [121]:
# Load model
model = read_sbml_model('models/iMM904_progesterone.xml')
model

Name,iMM904_progesterone
Memory address,0x07fb79d060d60
Number of metabolites,1234
Number of reactions,1586
Number of groups,0
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


In [122]:
# Progesterone is released from the cell
model.add_boundary(model.metabolites.get_by_id("progesterone_c"), type="demand")

Reaction identifier,DM_progesterone_c
Name,progesterone demand
Memory address,0x7fb79d060a00
Stoichiometry,progesterone_c --> progesterone -->
GPR,
Lower bound,0
Upper bound,1000.0


In [125]:
# Knockout ERG5 and ERG6
model_ERG5_ERG6 = model.copy()
model_ERG5_ERG6.genes.YMR015C.knock_out() # ERG5 knockout implemented
model_ERG5_ERG6.genes.YML008C.knock_out() # ERG6 knockout implemented

In [126]:
# check ERG5 knockout
model_ERG5_ERG6.genes.YMR015C

Gene identifier,YMR015C
Name,ERG5
Memory address,0x7fb78e2fb730
Functional,False
In 2 reaction(s),"C22STDS, C22STDSx"


In [127]:
# check ERG6 knockout
model_ERG5_ERG6.genes.YML008C

Gene identifier,YML008C
Name,ERG6
Memory address,0x7fb78e2ab760
Functional,False
In 1 reaction(s),SAM24MT


In [128]:
# ERG5 and ERG6 knockout influence on growth (default objective function)
print('Growth in model: ', model.optimize())
print('Growth when ERG5 and ERG6 knocked out: ', model_ERG5_ERG6.optimize())

Growth in model:  <Solution 0.288 at 0x7fb78f4c0be0>
Growth when ERG5 and ERG6 knocked out:  <Solution 0.288 at 0x7fb79233a670>


In [131]:
# ERG5 and ERG6 knockout influence on progesterone production (objective set to progesterone forming reactions)
model.objective = model.reactions.R02216
model_ERG5_ERG6.objective = model_ERG5_ERG6.reactions.R02216

print('Progesterone in model: ', round(model.optimize().objective_value,3))
print('Progesterone when ERG5 and ERG6 knocked out: ', round(model_ERG5_ERG6.optimize().objective_value,3))

Progesterone in model:  0.143
Progesterone when ERG5 and ERG6 knocked out:  0.143


Surprisingly, knocking out ERG5 and ERG6 does not seem to influence the cell growth or the progesterone production. We further investigate if this is the case or if there is a problem in the model to take into account.

In [132]:
# Investigate if the knockout of ERG5 and ERG6 influences other reactions
fluxes = model.optimize().fluxes
fluxes_knockout = model_ERG5_ERG6.optimize().fluxes
for i, (flux, flux_knockout) in enumerate(zip(fluxes, fluxes_knockout)):
    if flux != flux_knockout:
        print(f'Reaction {fluxes.index[i]}: flux: {flux} is different from knockout model flux: {flux_knockout}')

Reaction 2MBALDtm: flux: 0.0 is different from knockout model flux: 1.2857142857142858
Reaction 2MBTOHt: flux: 0.0 is different from knockout model flux: 1.2857142857142858
Reaction 2MBTOHtm: flux: 0.0 is different from knockout model flux: -1.2857142857142858
Reaction ACALDtm: flux: 0.0 is different from knockout model flux: -1.2857142857142858
Reaction ACCOAC: flux: 0.339517625231911 is different from knockout model flux: 0.0
Reaction 2OBUTtm: flux: 0.0 is different from knockout model flux: 1.2857142857142858
Reaction ACHBSm: flux: 0.0 is different from knockout model flux: 1.2857142857142858
Reaction ACLSm: flux: 0.2857142857142858 is different from knockout model flux: 0.0
Reaction AICART: flux: 0.1428571428571429 is different from knockout model flux: 0.0
Reaction AIRCr: flux: 0.1428571428571429 is different from knockout model flux: 0.0
Reaction ALCD22xim: flux: 0.0 is different from knockout model flux: 1.2857142857142858
Reaction ACS: flux: 2.9675324675324672 is different from

It is clear that it is not appropriate to knockout ERG5 and ERG6 for production progesterone. We investigate if knocking out other genes would improve the production of progesterone using OptGene from Cameo.

### References
Jiang, Yi-qi, and Jian-ping Lin. "Recent progress in strategies for steroid production in yeasts." World Journal of Microbiology and Biotechnology 38.6 (2022): 1-14.

Jordá, Tania, and Sergi Puig. "Regulation of ergosterol biosynthesis in Saccharomyces cerevisiae." Genes 11.7 (2020): 795.